In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from scipy import stats
from keras import backend as K
from keras.callbacks import TensorBoard, EarlyStopping
from sklearn.model_selection import StratifiedKFold
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import TimeDistributed
from keras.layers import Activation
from keras.layers import MaxPooling1D, UpSampling1D
from keras.layers import MaxPooling2D
from keras.layers import Flatten, Reshape
from keras.layers import Input, Dense, Bidirectional
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import Conv2D
from keras.layers import RepeatVector
from keras.layers.pooling import GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.optimizers import SGD
from keras.utils import to_categorical
from keras.utils import plot_model
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.models import Model
from keras.models import Sequential
import numpy as np
import os
import matplotlib.pyplot as plt
import random as rand
import pandas as pd
import tensorflow
import keras
import statistics as st
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
from keras.utils.vis_utils import plot_model
from sklearn.preprocessing import LabelEncoder
from collections import Counter

In [ ]:
os.chdir("/Users/Kush/Downloads/refullautoencodercode/")

In [ ]:
import p_f1_recall as metric
os.chdir("/Users/Kush/Documents/")

In [ ]:
results = []

In [ ]:
### HYPERPARAMETERS
batch_size, epochs, verbose = 200, 100, 1
encoding_dim = 64

In [ ]:
##### First proces unlabled data
data_u = pd.read_csv('unlabeled_data.csv')
print(list(data_u))
### Remove unncessary columns
data_u = data_u.iloc[:,[3,4,5,7,8,9]]  ## Add 13 if need to keep IDs  ## Drop heading because of low mutual information

### Standardize the features
def standardize(data,col_nums_to_standardize):
    for col in col_nums_to_standardize:
        data.iloc[:,col] = (data.iloc[:,col] - np.mean(data.iloc[:,col]))/np.std(data.iloc[:,col])
    return data

data_u = standardize(data_u,col_nums_to_standardize = range(6))

#### Now process the labeled data
data_train = pd.read_csv('final_training.csv')
data_test = pd.read_csv('final_testing.csv')

### Remove unncessary columns
data_train = data_train.iloc[:,[3,4,5,7,8,9,10]]  ## Add 13 if need to keep IDs ## Drop heading because of low mutual information
data_test = data_test.iloc[:,[3,4,5,7,8,9,10]]

### Standardize the features
data_train = standardize(data_train,col_nums_to_standardize = range(6))
data_test = standardize(data_test,col_nums_to_standardize = range(6))

['Unnamed: 0', 'COLLAR', 'TIME', 'LAT', 'LON', 'SPEED', 'HEADING', 'X_AXIS', 'Y_AXIS', 'Z_AXIS', 'ID']


In [ ]:
## Split the unlabeled data into training and validation
data_u_train, data_u_val = train_test_split(data_u,train_size=0.8,test_size=0.2,shuffle=True)

## Split the labeled data into training and validation
data_train, data_val = train_test_split(data_train,train_size=0.8,test_size=0.2,shuffle=True)

## Seperate features and labels and make labels one-hot
def split_fl(df):
    return np.array(df.iloc[:,:-1]), np.array(df.iloc[:,-1])

train_x, train_y = split_fl(data_train)
num_classes = len(set(train_y))
train_y = to_categorical(train_y)

val_x, val_y = split_fl(data_val)
val_y = to_categorical(val_y)
test_x, test_y = split_fl(data_test)
test_y = to_categorical(test_y)


def build_basicautoencoder(encoding_dim = encoding_dim): 
    raw_input = Input(shape=(data_u_train.shape[1],))
    
    encoding_0 = Dense(encoding_dim*6, activation='relu')(raw_input)
    encoding_1 = Dense(encoding_dim*4, activation='relu')(encoding_0)
    encoding_2 = Dense(encoding_dim*2, activation='relu')(encoding_1)
    encoded = Dense(encoding_dim, activation='relu')(encoding_2)
    decoding_1 = Dense(encoding_dim*2, activation='relu')(encoded)
    decoding_2 = Dense(encoding_dim*4, activation='relu')(decoding_1)
    decoding_0 = Dense(encoding_dim*6, activation='relu')(decoding_2)
    decoded = Dense(data_u_train.shape[1])(decoding_0)
    autoencoder = Model(raw_input, decoded)
    encoder = Model(raw_input, encoded)
    return autoencoder, encoder
    
autoencoder, encoder = build_basicautoencoder()
init_weights = encoder.get_weights()

def autoencoder_train(autoencoder):
    autoencoder.compile(optimizer='adam', loss='mse', metrics = ['mse'])
    autoencoder.fit(data_u_train, data_u_train,
                epochs=epochs,
                batch_size=batch_size,
                shuffle=True,
                validation_data=(data_u_val, data_u_val),
                callbacks=[EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto', baseline=None, restore_best_weights=True)])
    loss = autoencoder.evaluate(data_u_val, data_u_val, batch_size=batch_size, verbose=verbose)
    return loss

autoencoder_loss, autoencoder_loss = autoencoder_train(autoencoder)

trained_weights = encoder.get_weights()

Train on 719888 samples, validate on 179972 samples
Epoch 1/100
719888/719888 [==============================] - 55s 77us/step - loss: 0.0042 - mean_squared_error: 0.0042 - val_loss: 0.0016 - val_mean_squared_error: 0.0016
Epoch 2/100
719888/719888 [==============================] - 53s 73us/step - loss: 7.5851e-04 - mean_squared_error: 7.5851e-04 - val_loss: 2.8211e-04 - val_mean_squared_error: 2.8211e-04
Epoch 3/100
719888/719888 [==============================] - 71s 99us/step - loss: 5.6943e-04 - mean_squared_error: 5.6943e-04 - val_loss: 6.1829e-04 - val_mean_squared_error: 6.1829e-04
Epoch 4/100
719888/719888 [==============================] - 104s 144us/step - loss: 5.3232e-04 - mean_squared_error: 5.3232e-04 - val_loss: 0.0012 - val_mean_squared_error: 0.0012
Epoch 5/100
179972/179972 [==============================] - 10s 55us/step


In [ ]:
encoder.set_weights(trained_weights)
new_layer = Dense(num_classes, activation='softmax')(encoder.output)
pre_trained = Model(encoder.input,new_layer)
pre_trained.summary()

def train_model(model):
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy',metric.recall,metric.precision,metric.f1])
    model.fit(train_x,train_y,epochs=epochs,batch_size=batch_size,verbose=verbose,validation_data=(val_x,val_y),
             callbacks=[EarlyStopping(monitor='val_f1', min_delta=0, patience=3, verbose=0, mode='max', baseline=None, restore_best_weights=True)])
    loss,accuracy,recall_score,precision_score,f_score = model.evaluate(val_x, val_y, batch_size=batch_size, verbose=verbose)
    return loss,accuracy,recall_score,precision_score,f_score

m1_loss,m1_accuracy,m1_recall_score,m1_precision_score,m1_f_score = train_model(pre_trained)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_15 (InputLayer)        (None, 6)                 0         
_________________________________________________________________
dense_138 (Dense)            (None, 384)               2688      
_________________________________________________________________
dense_139 (Dense)            (None, 256)               98560     
_________________________________________________________________
dense_140 (Dense)            (None, 128)               32896     
_________________________________________________________________
dense_141 (Dense)            (None, 64)                8256      
_________________________________________________________________
dense_146 (Dense)            (None, 4)                 260       
Total params: 142,660
Trainable params: 142,660
Non-trainable params: 0
_________________________________________________________________
Trai

In [ ]:
### This shows how the model does using the same exact architecture without any pretraining
encoder.set_weights(init_weights)
new_layer = Dense(num_classes, activation='softmax')(encoder.output)
no_pre_train = Model(encoder.input,new_layer)
no_pre_train.summary()

m2_loss,m2_accuracy,m2_recall_score,m2_precision_score,m2_f_score = train_model(no_pre_train)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_15 (InputLayer)        (None, 6)                 0         
_________________________________________________________________
dense_138 (Dense)            (None, 384)               2688      
_________________________________________________________________
dense_139 (Dense)            (None, 256)               98560     
_________________________________________________________________
dense_140 (Dense)            (None, 128)               32896     
_________________________________________________________________
dense_141 (Dense)            (None, 64)                8256      
_________________________________________________________________
dense_147 (Dense)            (None, 4)                 260       
Total params: 142,660
Trainable params: 142,660
Non-trainable params: 0
_________________________________________________________________
Trai

In [ ]:
print("autoencoder_loss:",autoencoder_loss)
print("pretrained_f1_score:",m1_f_score)
print("no_pretrain_f1_score:",m2_f_score)

results.append([autoencoder_loss, m1_f_score, m2_f_score])

import os
os.system('say "Done"')
print(len(results))

autoencoder_loss: 0.0002821079630246951
pretrained_f1_score: 0.7184690528901618
no_pretrain_f1_score: 0.7711352526141885
5


In [ ]:
temp = np.array(results)
temp

array([[5.41914223e-04, 7.65763990e-01, 7.78080944e-01],
       [1.99469743e-04, 7.90588016e-01, 7.73855058e-01],
       [2.63841219e-04, 7.93403882e-01, 7.67219435e-01],
       [1.73083996e-04, 7.73571999e-01, 7.71384665e-01],
       [2.82107963e-04, 7.18469053e-01, 7.71135253e-01]])

In [ ]:
np.mean(temp, axis=0)

array([2.92083429e-04, 7.68359388e-01, 7.72335071e-01])

In [ ]:
np.std(temp, axis=0)

array([0.00013119, 0.02699738, 0.00357318])